# Testing Catastrophic forgetting with gym-MiniGrid trained over CNN policy

## Basic Jupyter Setting

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Import Stable-Baselines3 and initial set-up

Import libraries and classes:

In [3]:
# Imports
import time
import io
import os
import glob
import torch
import base64
import stable_baselines3

import numpy as np
import matplotlib.pyplot as plt

from random import randint 
from stable_baselines3 import PPO, DQN, A2C
from stable_baselines3.common.results_plotter import ts2xy, load_results
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

import gym
from gym import spaces
from gym.wrappers import Monitor
import gym_minigrid
from gym_minigrid.wrappers import FlatObsWrapper, ImgObsWrapper, RGBImgPartialObsWrapper

Define wrapper for CNN Policy

In [4]:
def ImgRGBImgPartialObsWrapper(env):
    return ImgObsWrapper(RGBImgPartialObsWrapper(env))

Define the video function:

In [5]:
import base64
import glob
import io
from IPython.display import HTML
from IPython import display 

def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")


Define the rendering wrappers:

In [7]:
from gym.wrappers import Monitor

# Monitor is a gym wrapper, which helps easy rendering of videos of the wrapped environment.
def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env

def gen_wrapped_env(env_name):
    return wrap_env(FlatObsWrapper(gym.make(env_name)))

def gen_wrapped_env_cnn(env_name):
    return wrap_env(ImgObsWrapper(RGBImgPartialObsWrapper(gym.make(env_name))))

# Test training with CNN Policy

## Define the environments

In [9]:
# By default, we use a DummyVecEnv as it is usually faster (cf doc)
num_cpu = 16  # Number of processes to use

env_id_1 = 'MiniGrid-DoorKey-6x6-v0'
vec_env_1 = make_vec_env(env_id_1, n_envs=num_cpu, wrapper_class=ImgRGBImgPartialObsWrapper, seed=10000)

env_id_2 = 'MiniGrid-LavaGapS6-v0'
vec_env_2 = make_vec_env(env_id_2, n_envs=num_cpu, wrapper_class=ImgRGBImgPartialObsWrapper, seed=5000)

Test which device is in use:

In [7]:
from stable_baselines3.common.utils import get_device
get_device()

device(type='cuda')

Test wrapper output shape:

In [8]:
learning_rate = 0.00005
n_steps = 256
batch_size = 16
ent_coef = 0.001
n_epochs = 4

tensorboard_log = "./tmp/log/"
os.makedirs(tensorboard_log, exist_ok=True)

## Learn first environment with CNN Policy

In [9]:
# Reset the environment
vec_env_1.reset()

# create the model
model = PPO('CnnPolicy',
            env=vec_env_1,
            learning_rate=learning_rate,
            batch_size=batch_size,
            ent_coef=ent_coef,
            n_epochs=n_epochs,
            n_steps=n_steps,
            tensorboard_log=tensorboard_log,
            verbose=1)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [10]:
log_dir = "./tmp/gym/"
os.makedirs(log_dir, exist_ok=True)
tb_log_name = 'task01-doorkey-6x6_cnn'

# Create eval environment
env = gym.make(env_id_1)
env = ImgRGBImgPartialObsWrapper(env)
eval_env = stable_baselines3.common.monitor.Monitor(env, log_dir)
# Reset the environment
eval_env.reset();

# Stop training when the model reaches the reward threshold
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=0.92, verbose=1)
eval_callback = EvalCallback(eval_env, log_path=log_dir, callback_on_new_best=callback_on_best, verbose=1)

In [11]:
total_timesteps = 1000000
log_interval = 10

model.learn(total_timesteps=total_timesteps,
            log_interval=log_interval,
            tb_log_name = tb_log_name,
            callback=eval_callback)

Logging to ./tmp/log/task01-doorkey-6x6_cnn_3


/home/inigo/stable-baselines3/stable_baselines3/common/callbacks.py:345: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x7f0d48500f10> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7f0ed0402ad0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 355         |
|    ep_rew_mean          | 0.0158      |
| time/                   |             |
|    fps                  | 282         |
|    iterations           | 10          |
|    time_elapsed         | 144         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.009526908 |
|    clip_fraction        | 0.0754      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.89       |
|    explained_variance   | -0.709      |
|    learning_rate        | 5e-05       |
|    loss                 | -0.0594     |
|    n_updates            | 36          |
|    policy_gradient_loss | -0.00487    |
|    value_loss           | 0.000583    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 358   

In [12]:
# Save the agent
model.save(tb_log_name)

In [13]:
#model = PPO.load(path=tb_log_name)

In [14]:
# We create a separate environment for evaluation
eval_env = ImgRGBImgPartialObsWrapper(gym.make(env_id_1))

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=20)
print(f'Mean reward: {mean_reward} +/- {std_reward:.2f}')

/home/inigo/stable-baselines3/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Mean reward: 0.866499987244606 +/- 0.29


## Learn second environment with CNN Policy

In [15]:
model.set_env(vec_env_2)

Wrapping the env in a VecTransposeImage.


In [10]:
log_dir = "./tmp/gym/"
os.makedirs(log_dir, exist_ok=True)
tb_log_name = 'task02-lava_cnn'

# Create eval environment
env = gym.make(env_id_1)
env = ImgRGBImgPartialObsWrapper(env)
eval_env = stable_baselines3.common.monitor.Monitor(env, log_dir)
# Reset the environment
eval_env.reset();

# Stop training when the model reaches the reward threshold
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=0.92, verbose=1)
eval_callback = EvalCallback(eval_env, log_path=log_dir, callback_on_new_best=callback_on_best, verbose=1)

In [17]:
# number of timesteps to add
total_timesteps = 2000000
log_interval = 10

# resume training using same tensorboard run without resetting the timesteps
model.learn(total_timesteps=total_timesteps,
            log_interval=log_interval,
            tb_log_name=tb_log_name,
            reset_num_timesteps=True,
            callback=eval_callback)

Logging to ./tmp/log/task02-lava_cnn_1


/home/inigo/stable-baselines3/stable_baselines3/common/callbacks.py:345: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x7f0d44e31a10> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7f0d4853f4d0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 31.1        |
|    ep_rew_mean          | 0.756       |
| time/                   |             |
|    fps                  | 280         |
|    iterations           | 10          |
|    time_elapsed         | 146         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.025103115 |
|    clip_fraction        | 0.234       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.96       |
|    explained_variance   | 0.906       |
|    learning_rate        | 5e-05       |
|    loss                 | -0.061      |
|    n_updates            | 348         |
|    policy_gradient_loss | -0.0114     |
|    value_loss           | 0.00614     |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 10.8  

In [18]:
# Save the agent
model.save(tb_log_name)

In [11]:
model = PPO.load(path=tb_log_name)

Evaluate training on second tast:

In [14]:
# We create a separate environment for evaluation
eval_env = ImgRGBImgPartialObsWrapper(gym.make(env_id_2))
eval_env.seed(10000+randint(0, 10))
eval_env.reset()
# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=20)
print(f'Mean reward: {mean_reward} +/- {std_reward:.2f}')

Mean reward: 0.5703125089406967 +/- 0.47


In [15]:
test_env = gen_wrapped_env_cnn(env_id_2)
test_env.seed(10000+randint(0, 10))
observation = test_env.reset()

done = False
episode_reward = 0
episode_length = 0

while not done:
  #test_env.render()
  action, states = model.predict(observation, deterministic=False)
  observation, reward, done, info = test_env.step(action)
  episode_reward += reward
  episode_length += 1

print('Total reward:', episode_reward)
print('Total length:', episode_length)

test_env.close()
show_video()

Total reward: 0.9375
Total length: 10


## Re-evaluate first task

In [16]:
# We create a separate environment for evaluation
eval_env = ImgRGBImgPartialObsWrapper(gym.make(env_id_1))
eval_env.seed(10000+randint(0, 10))
eval_env.reset()
# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=20)
print(f'Mean reward: {mean_reward} +/- {std_reward:.2f}')

/home/inigo/stable-baselines3/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Mean reward: 0.0 +/- 0.00
